<a href="https://colab.research.google.com/github/martinpius/Computer-Vission/blob/main/CNN_from_scratch_inception_architecture_for_mnist_dataset.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount("/content/drive/", force_remount = True)
try:
  COLAB = True
  import tensorflow as tf
  print(f"You are on CoLaB with tensorflow version: {tf.__version__}")
except Exception as e:
  print(f"{type(e)}: {e}\n...please load your drive...")
def time_fmt(x:float = 123.1789)->float:
  h = int(x / (60 * 60))
  m = int(x % (60 * 60) / 60)
  s = int(x % 60)
  return f"{h}: {m:>02}: {s:>05.2f}"
print(f"...time testing...time testing...time testing>>>:\ntime elapse: {time_fmt()}")

Mounted at /content/drive/
You are on CoLaB with tensorflow version: 2.4.1
...time testing...time testing...time testing>>>:
time elapse: 0: 02: 03.00


In [2]:
#In this notbook we are going to train a simple computer vission model with inception archtecture 
#on mnist dataset:

In [86]:
import tensorflow as tf
import tensorflow_datasets as tfds
import numpy as np
import time, sys, os

In [87]:
#We start by loading the mnist data from tensorflow datasets and prepare it for training:

In [88]:
(x_train, y_train),(x_test, y_test) = tf.keras.datasets.mnist.load_data()
    

In [89]:
x_train, x_test = x_train.reshape(-1,28,28,1).astype(np.float32)/255.0, x_test.reshape(-1,28,28,1).astype(np.float32)/255.0

In [90]:
y_train, y_test = tf.keras.utils.to_categorical(y_train, 10), tf.keras.utils.to_categorical(y_test, 10)

In [99]:
BATCH_SIZE = 128
BUFFER = len(x_train)
train_data = tf.data.Dataset.from_tensor_slices((x_train, y_train))
train_data = train_data.shuffle(BUFFER).batch(BATCH_SIZE, drop_remainder = True)
test_data = tf.data.Dataset.from_tensor_slices((x_test, y_test))
test_data = test_data.batch(BATCH_SIZE, drop_remainder = True)


In [92]:
#Now our data is ready for training: Lets build our model:

In [93]:
#Convolutional block:
class CNNBLOCK(tf.keras.layers.Layer):
  def __init__(self,num_filters, kernel = 3, *args, **kwargs):
    super(CNNBLOCK, self).__init__(*args, **kwargs)
    self.conv = tf.keras.layers.Conv2D(filters = num_filters, 
                                       kernel_size = kernel,
                                       kernel_initializer = 'random_normal',
                                       strides = 1,padding = 'same')
    self.bn = tf.keras.layers.BatchNormalization()
    self.myact = tf.keras.layers.LeakyReLU(alpha = 0.3)
  def call(self, inputs_tensor , training = False):
    x = self.conv(inputs_tensor, training = training)
    x = self.myact(x)
    x = self.bn(x)
    return x

In [94]:
#Residual Block:
class ResNetBlock(tf.keras.layers.Layer):
  def __init__(self, num_filters, *args, **kwargs):
    super(ResNetBlock, self).__init__(*args, **kwargs)
    self.block1 = CNNBLOCK(num_filters = num_filters[0], name = 'bl_1')
    self.block2 = CNNBLOCK(num_filters = num_filters[1], name = 'bl_2')
    self.block3 = CNNBLOCK(num_filters = num_filters[2], name = 'bl_3')
    self.max_pool = tf.keras.layers.MaxPooling2D()
    self.id_mapping = tf.keras.layers.Conv2D(filters = num_filters[1], kernel_size = 3, padding = 'same',
                                             activation = 'relu', kernel_initializer = 'random_normal')
  def call(self, inputs_tensor, training = False):
    x = self.block1(inputs_tensor, training = training)
    x = self.block2(x, training = training)
    x = self.block3(x + self.id_mapping(inputs_tensor), training = training)
    return self.max_pool(x)

In [95]:
#Now we build a simple resnet model using model subclassing:

In [96]:
class ResModel(tf.keras.Model):
  def __init__(self, classes = 10, *args, **kwargs):
    super(ResModel, self).__init__(*args, **kwargs)
    self.resblock1 = ResNetBlock(num_filters = [32,64,128], name = 'resbl_1')
    self.resblock2 = ResNetBlock(num_filters = [128,256,512], name = 'resbl_2')
    self.resblock3 = ResNetBlock(num_filters = [64,128,128], name = 'resbl_3')
    self.glb = tf.keras.layers.Flatten()
    self.dense1 = tf.keras.layers.Dense(units = 1024, activation = 'relu',kernel_initializer = 'random_normal')
    self.dense2 = tf.keras.layers.Dense(units = 128, activation = 'relu', kernel_initializer = 'random_normal')
    self.outputs = tf.keras.layers.Dense(units = 10, name = 'outputs') 
  def call(self, inputs_tensor ,training = False):
    x = self.resblock1(inputs_tensor, training = training)
    x = self.resblock2(x, training = training)
    x = self.resblock3(x, training = training)
    x = self.glb(x)
    x = self.dense1(x, training = training)
    x = self.dense2(x, training = training)
    x = self.outputs(x, training = training)
    return x


In [97]:
#We can now instatiating our model ready for training:

In [100]:
model = ResModel()

tic = time.time()
model.compile(loss = tf.keras.losses.CategoricalCrossentropy(from_logits = True),
              optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001),
              metrics = ['accuracy'])
model.fit(train_data, epochs = 3, verbose = 2)
model.evaluate(test_data, verbose = 2)
toc = time.time()
print(model.summary())
print(f"\ntime elapse: {time_fmt(toc - tic)}")


Epoch 1/3
468/468 - 37s - loss: 0.1913 - accuracy: 0.9548
Epoch 2/3
468/468 - 36s - loss: 0.0424 - accuracy: 0.9870
Epoch 3/3
468/468 - 36s - loss: 0.0286 - accuracy: 0.9910
78/78 - 2s - loss: 0.0251 - accuracy: 0.9928
Model: "res_model_16"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
resbl_1 (ResNetBlock)        multiple                  94208     
_________________________________________________________________
resbl_2 (ResNetBlock)        multiple                  1921664   
_________________________________________________________________
resbl_3 (ResNetBlock)        multiple                  1107648   
_________________________________________________________________
flatten_4 (Flatten)          multiple                  0         
_________________________________________________________________
dense_32 (Dense)             multiple                  1180672   
_________________________________

In [101]:
#The training loop:
EPOCHS = 30
tic = time.time()
loss_obj = tf.keras.losses.CategoricalCrossentropy(from_logits = True)
optimizer = tf.keras.optimizers.Adam(learning_rate = 0.001)
train_metric = tf.keras.metrics.CategoricalAccuracy()
val_metric = tf.keras.metrics.CategoricalAccuracy()
for epoch in range(EPOCHS):
  print(f"training starts at epoch: {epoch + 1}\ntraining::>>>please wait....")
  for (step, (x_train_batch, y_train_batch)) in enumerate(train_data):
    with tf.GradientTape() as tape:
      preds = model(x_train_batch, training = True)
      train_loss = loss_obj(y_train_batch, preds)
    grads = tape.gradient(train_loss, model.trainable_weights)
    optimizer.apply_gradients(zip(grads, model.trainable_weights))
    train_metric.update_state(y_train_batch, preds)
    train_acc = train_metric.result()
    if step % 200 == 0:
      print(f"epoch: {epoch + 1}, train accuracy: {float(train_acc):.4f}")
      print(f"batch number: {step}, train loss: {float(train_loss):.4f}")
  for (step, (x_val_batch, y_val_batch)) in enumerate(test_data):
    preds = model(x_val_batch, training = False)
    val_loss = loss_obj(y_val_batch, preds)
    val_metric.update_state(y_val_batch, preds)
    val_acc = val_metric.result()
    val_metric.reset_states()
    if step % 200 == 0:
      print(f"epoch: {epoch + 1}, validation accuracy: {float(val_acc):.4f}")
      print(f"batch number: {step}, validation loss: {float(val_loss):.4f}")
toc = time.time()
print(model.summary())
print(f"\ntotal time for training and evaluation: {time_fmt(toc - tic)}")

training starts at epoch: 1
training::>>>please wait....
epoch: 1, train accuracy: 0.9922
batch number: 0, train loss: 0.0116
epoch: 1, train accuracy: 0.9835
batch number: 200, train loss: 0.0410
epoch: 1, train accuracy: 0.9858
batch number: 400, train loss: 0.0434
epoch: 1, validation accuracy: 0.9922
batch number: 0, validation loss: 0.0212
training starts at epoch: 2
training::>>>please wait....
epoch: 2, train accuracy: 0.9863
batch number: 0, train loss: 0.0222
epoch: 2, train accuracy: 0.9877
batch number: 200, train loss: 0.0084
epoch: 2, train accuracy: 0.9885
batch number: 400, train loss: 0.0160
epoch: 2, validation accuracy: 0.9922
batch number: 0, validation loss: 0.0331
training starts at epoch: 3
training::>>>please wait....
epoch: 3, train accuracy: 0.9889
batch number: 0, train loss: 0.0252
epoch: 3, train accuracy: 0.9899
batch number: 200, train loss: 0.0019
epoch: 3, train accuracy: 0.9904
batch number: 400, train loss: 0.0296
epoch: 3, validation accuracy: 0.9922
